# 📊 Reporte de Pull Requests Abiertos en Bitbucket

Este reporte proporciona una vista completa de todos los pull requests que se encuentran abiertos en los repositorios de Bitbucket, incluyendo información detallada sobre el estado, reviewers, comentarios y tiempos de apertura para una mejor gestión y seguimiento.

## 🔧 Configuración e Importación de Librerías

In [ ]:
%pip install python-dotenv requests pandas matplotlib seaborn

import requests
import pandas as pd
from dotenv import load_dotenv
import os
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings("ignore")

print("✅ Librerías importadas correctamente")

In [ ]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Configurar credenciales de Bitbucket
username = os.getenv("BITBUCKET_USERNAME")
app_password = os.getenv("BITBUCKET_APP_PASSWORD")
workspace = os.getenv("BITBUCKET_WORKSPACE")

# Verificar que las credenciales estén configuradas
if not all([username, app_password, workspace]):
    print("❌ Error: Asegúrate de tener configuradas las variables de entorno:")
    print("- BITBUCKET_USERNAME")
    print("- BITBUCKET_APP_PASSWORD")
    print("- BITBUCKET_WORKSPACE")
else:
    print(f"✅ Credenciales configuradas correctamente para el workspace: {workspace}")
    print(f"   Usuario: {username}")

## 📦 Obtener Repositorios y Funciones de Utilidad

In [ ]:
# Función para obtener todos los elementos paginados de la API de Bitbucket
def get_all_items(url, params=None):
    """
    Obtiene todos los elementos de una URL paginada de la API de Bitbucket
    """
    items = []
    while url:
        response = requests.get(url, params=params, auth=(username, app_password))
        if response.status_code != 200:
            print(
                f"❌ Error al obtener datos: {response.status_code} - {response.text}"
            )
            break

        data = response.json()
        items.extend(data.get("values", []))

        # Obtener la siguiente página si existe
        url = data.get("next")
        params = None  # Los parámetros ya están incluidos en la URL 'next'

    return items


# Función para obtener comentarios de un pull request
def get_pr_comments(workspace, repo_slug, pr_id):
    """
    Obtiene todos los comentarios de un pull request específico
    """
    comments_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests/{pr_id}/comments"
    return get_all_items(comments_url)


print("✅ Funciones de utilidad definidas")

In [ ]:
# Obtener lista de repositorios
url_repos = f"https://api.bitbucket.org/2.0/repositories/{workspace}"

print("🔄 Obteniendo lista de repositorios...")
repositorios_data = get_all_items(url_repos)

if repositorios_data:
    repos_list = []
    for repo in repositorios_data:
        repos_list.append(
            {
                "Nombre": repo.get("name"),
                "URL": repo.get("links", {}).get("html", {}).get("href"),
                "Descripción": repo.get("description", ""),
                "Privado": repo.get("is_private", False),
            }
        )

    repos_df = pd.DataFrame(repos_list)
    repos_df = repos_df.sort_values(by="Nombre")

    # Filtrar repositorios (excluir algunos como en el archivo original)
    repositorios_excluir = ["pgp", "Pruebas_erp", "Inventario", "b2c", "efi"]
    repositorios_activos = [
        repo["Nombre"]
        for repo in repos_list
        if repo["Nombre"] not in repositorios_excluir
    ]

    print(f"✅ Total de repositorios encontrados: {len(repositorios_data)}")
    print(f"✅ Repositorios activos para análisis: {len(repositorios_activos)}")

    # Mostrar algunos repositorios
    display(repos_df.head(10))
else:
    print("❌ No se pudieron obtener los repositorios")

## 🔍 Obtener Pull Requests Abiertos

In [ ]:
# Obtener todos los pull requests abiertos
print("🔄 Obteniendo pull requests abiertos de todos los repositorios...")

pull_requests_abiertos = []
total_repos = len(repositorios_activos)

for i, repo_slug in enumerate(repositorios_activos, 1):
    print(f"   Procesando repositorio {i}/{total_repos}: {repo_slug}")

    # URL para obtener PRs abiertos
    pr_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests"

    # Obtener solo PRs abiertos
    params = {"state": "OPEN"}
    prs = get_all_items(pr_url, params=params)

    # Agregar información del repositorio a cada PR
    for pr in prs:
        pr["repository"] = repo_slug

    pull_requests_abiertos.extend(prs)

print(
    f"\n✅ Total de pull requests abiertos encontrados: {len(pull_requests_abiertos)}"
)

if len(pull_requests_abiertos) == 0:
    print("ℹ️  No hay pull requests abiertos en este momento")
else:
    print("🔄 Procesando información adicional de los pull requests...")

In [ ]:
# Procesar información detallada de cada pull request
if len(pull_requests_abiertos) > 0:
    prs_procesados = []

    for i, pr in enumerate(pull_requests_abiertos, 1):
        print(
            f"   Procesando PR {i}/{len(pull_requests_abiertos)}: {pr.get('title', 'Sin título')[:50]}..."
        )

        # Obtener comentarios del PR
        try:
            comentarios = get_pr_comments(workspace, pr["repository"], pr["id"])
            total_comentarios = len(comentarios)

            # Contar comentarios sin resolver (aquellos que no tienen respuesta o resolución)
            comentarios_sin_resolver = 0
            for comentario in comentarios:
                # Un comentario se considera sin resolver si no tiene resolved=True
                if not comentario.get("resolved", False):
                    comentarios_sin_resolver += 1
        except Exception as e:
            print(f"      ⚠️  Error obteniendo comentarios: {str(e)}")
            total_comentarios = 0
            comentarios_sin_resolver = 0

        # Extraer información de reviewers
        reviewers = []
        if "reviewers" in pr and pr["reviewers"]:
            for reviewer in pr["reviewers"]:
                if "nickname" in reviewer:
                    reviewers.append(reviewer["nickname"])
                elif "display_name" in reviewer:
                    reviewers.append(reviewer["display_name"])

        # Calcular días abierto
        created_on = pd.to_datetime(pr["created_on"])
        now = pd.Timestamp.now(tz="UTC")
        dias_abierto = (now - created_on).days

        # Extraer información del autor
        autor = "Desconocido"
        if "author" in pr and pr["author"]:
            if "nickname" in pr["author"]:
                autor = pr["author"]["nickname"]
            elif "display_name" in pr["author"]:
                autor = pr["author"]["display_name"]

        # Crear registro procesado
        pr_procesado = {
            "ID": pr["id"],
            "Título": pr.get("title", "Sin título"),
            "Repositorio": pr["repository"],
            "Autor": autor,
            "Reviewers": ", ".join(reviewers) if reviewers else "Sin reviewers",
            "Cantidad_Reviewers": len(reviewers),
            "Total_Comentarios": total_comentarios,
            "Comentarios_Sin_Resolver": comentarios_sin_resolver,
            "Fecha_Creacion": created_on.strftime("%Y-%m-%d %H:%M"),
            "Ultima_Actualizacion": pd.to_datetime(pr["updated_on"]).strftime(
                "%Y-%m-%d %H:%M"
            ),
            "Dias_Abierto": dias_abierto,
            "Descripcion": pr.get("description", "Sin descripción")[:100] + "..."
            if pr.get("description", "")
            else "Sin descripción",
            "URL": pr.get("links", {}).get("html", {}).get("href", ""),
            "Rama_Origen": pr.get("source", {})
            .get("branch", {})
            .get("name", "Desconocida"),
            "Rama_Destino": pr.get("destination", {})
            .get("branch", {})
            .get("name", "Desconocida"),
        }

        prs_procesados.append(pr_procesado)

    # Crear DataFrame con los datos procesados
    df_prs_abiertos = pd.DataFrame(prs_procesados)

    # Ordenar por días abierto (descendente) para mostrar los más antiguos primero
    df_prs_abiertos = df_prs_abiertos.sort_values("Dias_Abierto", ascending=False)

    print(
        f"\n✅ Procesamiento completado. {len(df_prs_abiertos)} pull requests procesados"
    )
else:
    df_prs_abiertos = pd.DataFrame()

## 📊 Reportes y Análisis

In [ ]:
# Resumen ejecutivo
if not df_prs_abiertos.empty:
    print("=" * 60)
    print("📈 RESUMEN EJECUTIVO - PULL REQUESTS ABIERTOS")
    print("=" * 60)

    total_prs = len(df_prs_abiertos)
    prs_con_reviewers = len(df_prs_abiertos[df_prs_abiertos["Cantidad_Reviewers"] > 0])
    prs_sin_reviewers = total_prs - prs_con_reviewers

    prs_con_comentarios = len(df_prs_abiertos[df_prs_abiertos["Total_Comentarios"] > 0])
    total_comentarios = df_prs_abiertos["Total_Comentarios"].sum()
    total_comentarios_sin_resolver = df_prs_abiertos["Comentarios_Sin_Resolver"].sum()

    promedio_dias_abierto = df_prs_abiertos["Dias_Abierto"].mean()
    max_dias_abierto = df_prs_abiertos["Dias_Abierto"].max()

    print(f"🔢 Total de Pull Requests Abiertos: {total_prs}")
    print(
        f"📝 PRs con Reviewers Asignados: {prs_con_reviewers} ({prs_con_reviewers / total_prs * 100:.1f}%)"
    )
    print(
        f"⚠️  PRs sin Reviewers: {prs_sin_reviewers} ({prs_sin_reviewers / total_prs * 100:.1f}%)"
    )
    print(f"💬 PRs con Comentarios: {prs_con_comentarios}")
    print(f"📊 Total de Comentarios: {total_comentarios}")
    print(f"❗ Comentarios Sin Resolver: {total_comentarios_sin_resolver}")
    print(f"⏱️  Promedio de Días Abierto: {promedio_dias_abierto:.1f} días")
    print(f"🚨 PR más Antiguo: {max_dias_abierto} días")

    # Repositorios con más PRs abiertos
    repos_prs = df_prs_abiertos["Repositorio"].value_counts()
    print(f"\n🏆 Repositorios con más PRs abiertos:")
    for i, (repo, count) in enumerate(repos_prs.head(5).items(), 1):
        print(f"   {i}. {repo}: {count} PRs")

    # Autores con más PRs abiertos
    autores_prs = df_prs_abiertos["Autor"].value_counts()
    print(f"\n👥 Autores con más PRs abiertos:")
    for i, (autor, count) in enumerate(autores_prs.head(5).items(), 1):
        print(f"   {i}. {autor}: {count} PRs")

    print("=" * 60)
else:
    print("ℹ️  No hay pull requests abiertos para mostrar en el resumen")

In [ ]:
# Mostrar tabla detallada de Pull Requests
if not df_prs_abiertos.empty:
    print("\n📋 DETALLE DE PULL REQUESTS ABIERTOS")
    print("=" * 60)

    # Crear una versión de la tabla optimizada para visualización
    df_display = df_prs_abiertos[
        [
            "ID",
            "Repositorio",
            "Título",
            "Autor",
            "Reviewers",
            "Cantidad_Reviewers",
            "Total_Comentarios",
            "Comentarios_Sin_Resolver",
            "Dias_Abierto",
            "Fecha_Creacion",
            "Ultima_Actualizacion",
        ]
    ].copy()

    # Configurar pandas para mostrar todas las columnas
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", None)
    pd.set_option("display.max_colwidth", 50)

    # Mostrar la tabla
    display(df_display)

    print(f"\n📌 Mostrando {len(df_display)} pull requests abiertos")
    print("💡 Tip: Los PRs están ordenados por días abierto (más antiguos primero)")
else:
    print("ℹ️  No hay pull requests abiertos para mostrar")

In [ ]:
# Visualizaciones
if not df_prs_abiertos.empty:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # 1. PRs por repositorio
    repos_count = df_prs_abiertos["Repositorio"].value_counts().head(10)
    axes[0, 0].bar(range(len(repos_count)), repos_count.values, color="skyblue")
    axes[0, 0].set_title("PRs Abiertos por Repositorio (Top 10)")
    axes[0, 0].set_xlabel("Repositorio")
    axes[0, 0].set_ylabel("Cantidad de PRs")
    axes[0, 0].set_xticks(range(len(repos_count)))
    axes[0, 0].set_xticklabels(repos_count.index, rotation=45, ha="right")

    # 2. Distribución de días abierto
    axes[0, 1].hist(
        df_prs_abiertos["Dias_Abierto"], bins=20, color="lightcoral", alpha=0.7
    )
    axes[0, 1].set_title("Distribución de Días Abierto")
    axes[0, 1].set_xlabel("Días Abierto")
    axes[0, 1].set_ylabel("Cantidad de PRs")
    axes[0, 1].axvline(
        df_prs_abiertos["Dias_Abierto"].mean(),
        color="red",
        linestyle="--",
        label=f"Promedio: {df_prs_abiertos['Dias_Abierto'].mean():.1f} días",
    )
    axes[0, 1].legend()

    # 3. PRs con y sin reviewers
    reviewers_data = (
        df_prs_abiertos["Cantidad_Reviewers"]
        .apply(lambda x: "Con Reviewers" if x > 0 else "Sin Reviewers")
        .value_counts()
    )
    axes[1, 0].pie(
        reviewers_data.values,
        labels=reviewers_data.index,
        autopct="%1.1f%%",
        colors=["lightgreen", "lightcoral"],
    )
    axes[1, 0].set_title("PRs: Con vs Sin Reviewers")

    # 4. Comentarios sin resolver por PR
    prs_con_comentarios = df_prs_abiertos[df_prs_abiertos["Total_Comentarios"] > 0]
    if not prs_con_comentarios.empty:
        axes[1, 1].scatter(
            prs_con_comentarios["Total_Comentarios"],
            prs_con_comentarios["Comentarios_Sin_Resolver"],
            alpha=0.6,
            color="orange",
        )
        axes[1, 1].set_title("Comentarios Total vs Sin Resolver")
        axes[1, 1].set_xlabel("Total de Comentarios")
        axes[1, 1].set_ylabel("Comentarios Sin Resolver")

        # Línea diagonal para mostrar cuando todos los comentarios están sin resolver
        max_comments = max(
            prs_con_comentarios["Total_Comentarios"].max(),
            prs_con_comentarios["Comentarios_Sin_Resolver"].max(),
        )
        axes[1, 1].plot(
            [0, max_comments],
            [0, max_comments],
            "r--",
            alpha=0.5,
            label="Todos sin resolver",
        )
        axes[1, 1].legend()
    else:
        axes[1, 1].text(
            0.5,
            0.5,
            "No hay PRs\ncon comentarios",
            horizontalalignment="center",
            verticalalignment="center",
            transform=axes[1, 1].transAxes,
            fontsize=12,
        )
        axes[1, 1].set_title("Comentarios Total vs Sin Resolver")

    plt.tight_layout()
    plt.show()
else:
    print("📊 No hay datos para generar gráficos")

In [ ]:
# Alertas y recomendaciones para gestión
if not df_prs_abiertos.empty:
    print("\n🚨 ALERTAS Y RECOMENDACIONES")
    print("=" * 60)

    # PRs antiguos (más de 7 días)
    prs_antiguos = df_prs_abiertos[df_prs_abiertos["Dias_Abierto"] > 7]
    if not prs_antiguos.empty:
        print(f"⏰ PRs ANTIGUOS (más de 7 días): {len(prs_antiguos)}")
        for _, pr in prs_antiguos.head(5).iterrows():
            print(
                f"   • {pr['Repositorio']}: '{pr['Título'][:40]}...' - {pr['Dias_Abierto']} días"
            )
        if len(prs_antiguos) > 5:
            print(f"   ... y {len(prs_antiguos) - 5} más")

    # PRs sin reviewers
    prs_sin_reviewers = df_prs_abiertos[df_prs_abiertos["Cantidad_Reviewers"] == 0]
    if not prs_sin_reviewers.empty:
        print(f"\n👥 PRs SIN REVIEWERS: {len(prs_sin_reviewers)}")
        for _, pr in prs_sin_reviewers.head(5).iterrows():
            print(f"   • {pr['Repositorio']}: '{pr['Título'][:40]}...' - {pr['Autor']}")
        if len(prs_sin_reviewers) > 5:
            print(f"   ... y {len(prs_sin_reviewers) - 5} más")

    # PRs con muchos comentarios sin resolver
    prs_comentarios_problema = df_prs_abiertos[
        df_prs_abiertos["Comentarios_Sin_Resolver"] >= 3
    ]
    if not prs_comentarios_problema.empty:
        print(
            f"\n💬 PRs CON MUCHOS COMENTARIOS SIN RESOLVER (3+): {len(prs_comentarios_problema)}"
        )
        for _, pr in prs_comentarios_problema.head(5).iterrows():
            print(
                f"   • {pr['Repositorio']}: '{pr['Título'][:40]}...' - {pr['Comentarios_Sin_Resolver']} sin resolver"
            )
        if len(prs_comentarios_problema) > 5:
            print(f"   ... y {len(prs_comentarios_problema) - 5} más")

    print(f"\n💡 RECOMENDACIONES:")
    print(f"   • Revisar PRs antiguos para acelerar el proceso de merge")
    print(f"   • Asignar reviewers a PRs que no los tienen")
    print(f"   • Resolver comentarios pendientes para destrabar PRs")
    print(f"   • Considerar establecer SLAs para tiempo máximo de PRs abiertos")

    print("=" * 60)
else:
    print("✅ ¡Excelente! No hay pull requests abiertos que requieran atención")

In [ ]:
# Exportar datos para seguimiento
if not df_prs_abiertos.empty:
    print("\n💾 EXPORTACIÓN DE DATOS")
    print("=" * 40)

    # Generar nombre de archivo con timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"pull_requests_abiertos_{timestamp}.csv"

    try:
        # Exportar a CSV
        df_prs_abiertos.to_csv(filename, index=False, encoding="utf-8-sig")
        print(f"✅ Datos exportados a: {filename}")

        # Mostrar resumen de la exportación
        print(
            f"📊 Archivo contiene {len(df_prs_abiertos)} registros con las siguientes columnas:"
        )
        for col in df_prs_abiertos.columns:
            print(f"   • {col}")

    except Exception as e:
        print(f"❌ Error al exportar: {str(e)}")

    print("=" * 40)

    # Información para próxima ejecución
    print(f"\n🔄 PRÓXIMA EJECUCIÓN")
    print(f"   • Ejecutar este notebook regularmente para monitorear PRs")
    print(f"   • Fecha/hora actual: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"   • Recomendación: Ejecutar diariamente o semanalmente")
else:
    print("ℹ️  No hay datos para exportar")